In [0]:
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/Modulo de Seguimiento/')

import SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs,greatest
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

#Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

from collections import Counter

def find_repeated_columns(df):
    column_names = df.columns
    column_counts = Counter(column_names)
    repeated_columns = [col for col, count in column_counts.items() if count > 1]
    return repeated_columns

def find_missing_columns(df1, df2):
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)
    
    missing_in_df2 = columns_df1 - columns_df2
    missing_in_df1 = columns_df2 - columns_df1
    
    print("Columnas en df1 pero no en df2:", missing_in_df2)
    print("Columnas en df2 pero no en df1:", missing_in_df1)

In [0]:
hip = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_univ_pre_driver_app_hip')

gahi = hip.withColumn(
    "FLG_FINALIDAD",
    when(col("DESTIPFINALIDADCREDHIPOTECARIO").isNull(), lit(1))
    .when(col("DESTIPFINALIDADCREDHIPOTECARIO").isin(
        "LIBRE DISPONIBILIDAD",
        "COMPRA VENTA DE LOCAL COMERCIAL",
        "C-V/REMODEL/AMPLI DE LOCAL COMERCIAL",
        "C-V Oficinas/Locales BT",
        "LIBRE DISP. CON GARANT.LOCAL COMERC.",
        "MIGRACION DE OTRO BANCO"
    ), lit(1))
    .when((col("CODMES") >= 202307) & (col("DESTIPFINALIDADCREDHIPOTECARIO") == "COMPRA/VENTA BIEN TERMINADO"), lit(1))
    .otherwise(lit(0))
).filter(col("TIPCREDITO").isin('GAHI', 'GAHP', 'GAHR', 'GAPC'))

## Base espejo

In [0]:
espejo_total = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_mm_appscore_gahi')
gahi.createOrReplaceTempView('gahi')
espejo_total.createOrReplaceTempView('espejo_total')
gahi_2 = spark.sql("""
    SELECT A.*, 
           /*VARIABLES CALIBRADO*/
           D.XB_GAHI,
           D.XB_GAHI_PRELIMINAR,
           D.XB_NUE_APP_CEF, /*89%*/
           D.XB_MODUL_GAHI, /*11%*/
           D.LTV_NEW AS LTV_NEW_CAL,
           D.LTV_NEW_A2,
           D.CTDPLAZOAPROBADO_TP,
           D.CTDPLAZOAPROBADO_A2,
           D.RIESGO_ZONA2,
           D.RIESGO_ZONA2_ST,
           D.INGRESO_SOL_CONY_TP,
           D.INGRESO_SOL_CONY_D3, 
           D.FLG_MONTO AS FLG_MONTO_CAL, /*SI >=325,000 FLG_MONTO=1*/
           D.PD_GAHI AS PD_GAHI_ESPEJO
    FROM gahi A
    LEFT JOIN espejo_total D ON A.CODSOLICITUD=D.CODSOLICITUD;
""")

In [0]:
# Add the TOTAL column
gahi_2 = gahi_2.withColumn("TOTAL", lit('1'))

# Add the PAUTA column
gahi_2 = gahi_2.withColumn("PAUTA", when(col("SC_GAHI") >= 345, 1).otherwise(0))

# Add the RSC column
gahi_2 = gahi_2.withColumn("RSC", when(col("SC_GAHI").isNull(), '99. Missing')
                   .when(col("SC_GAHI") < 385, '1. <0 - 385>')
                   .when(col("SC_GAHI") < 415, '2. [385 - 415>')
                   .when(col("SC_GAHI") < 450, '3. [415 - 450>')
                   .when(col("SC_GAHI") < 495, '4. [450 - 495>')
                   .when(col("SC_GAHI") >= 495, '5. [495 - +++>')
                   .otherwise('98. Otro'))

# Add the RPD column
gahi_2 = gahi_2.withColumn("RPD", when(col("PD_GAHI").isNull(), '99. Missing')
                   .when(col("PD_GAHI") <= 0.005, '1. <0 - 0.5%]')
                   .when(col("PD_GAHI") <= 0.010, '2. <0.5 - 1.0%]')
                   .when(col("PD_GAHI") <= 0.015, '3. <1.0 - 1.5%]')
                   .when(col("PD_GAHI") <= 0.025, '4. <1.5 - 2.5%]')
                   .when(col("PD_GAHI") > 0.025, '5. <2.5 - ++>')
                   .otherwise('98. Otro'))

# Add the MONTO_R column
gahi_2 = gahi_2.withColumn("MONTO_R", when(col("MTOAPROBADO_SOLES").isNull(), '99. Missing')
                   .when(col("MTOAPROBADO_SOLES") <= 87500, '1. [0 - 87,500]')
                   .when(col("MTOAPROBADO_SOLES") <= 140000, '2. <87,500 - 140,000]')
                   .when(col("MTOAPROBADO_SOLES") <= 215000, '3. <140,000 - 215,000]')
                   .when(col("MTOAPROBADO_SOLES") <= 300000, '4. <215,000 - 300,000]')
                   .when(col("MTOAPROBADO_SOLES") <= 600000, '5. <300,000 - 600,000]')
                   .otherwise('6. <600,000 - ++>'))

# Add the MISS_PD column
gahi_2 = gahi_2.withColumn("MISS_PD", when(col("PD_GAHI").isNull(), 1).otherwise(0))

# Add the MISS_MONTO column
gahi_2 = gahi_2.withColumn("MISS_MONTO", when(col("MTOAPROBADO_SOLES").isNull(), 1).otherwise(0))

# Add the RAN_INGR_CY column
gahi_2 = gahi_2.withColumn("RAN_INGR_CY", when(col("INGRESO_SOL_CONY_TP").isNull(), '99. Missing')
                   .when(col("INGRESO_SOL_CONY_TP") <= 2500, '1. [0 - 2,500]')
                   .when(col("INGRESO_SOL_CONY_TP") <= 5000, '2. <2,500 - 5,000]')
                   .when(col("INGRESO_SOL_CONY_TP") <= 7500, '3. <5,000 - 7,500]')
                   .when(col("INGRESO_SOL_CONY_TP") <= 10000, '4. <7,500 - 10,000]')
                   .when(col("INGRESO_SOL_CONY_TP") <= 20000, '5. <10,000 - 20,000]')
                   .otherwise('6. <20,000 - ++>'))

# Add the FLG_MONTO_R column
gahi_2 = gahi_2.withColumn("FLG_MONTO_R", when(col("FLG_MONTO_CAL").isNull(), '99. Missing')
                   .when(col("FLG_MONTO_CAL") == 1, '1. >=325,000')
                   .otherwise('2. <325,000'))

# Filter the dataframe
gahi_2 = gahi_2.filter(col("FLG_FINALIDAD") == 1)

In [0]:
gahi_2.createOrReplaceTempView("hip_app_gahi")

spark.sql('''
    CREATE OR REPLACE TABLE
        catalog_lhcl_prod_bcp.bcp_edv_fabseg.T39290_hm_app_gahi
    USING DELTA LOCATION 
        'abfss://bcp-edv-fabseg@adlscu1lhclbackp05.dfs.core.windows.net/T39290/Seguimiento/02_Applicant_Hipotecario/hm_app_gahi'
    AS SELECT * FROM hip_app_gahi
    '''
)